In [55]:
# from pathlib import Path

import pandas as pd
from am_notebooks import *

from source.utils import HIT_TABLES_DIR, print_iter, timestamp_today
from source.utils.associate import AM_DF_DIR, TOP_AM_DIR, adjust_am_names as adjust_assoc_columns 
from source.utils.dataframes import catify_hit_table as catify

# from source.utils.sample import sample_pickle as sp

TAG='ALL'
K=8
BK = max(K+2, 10)
BIGRAM_F_FLOOR=50 if TAG == 'ALL' else 25

ADV_F_FLOOR=5000

# METRIC_PRIORITY = ['LRC', 'P1', 'G2', 'P2'] if TAG=='NEQ' else ['dP1', 'LRC', 'G2', 'P1']
TOP_AM_TAG_DIR = TOP_AM_DIR / TAG
TAG_TOP_STR = f'{TAG}-Top{K}'
TAG_TOP_DIR = TOP_AM_TAG_DIR / TAG_TOP_STR
DATE=timestamp_today()
FOCUS_ORIG = ['f', 'E11', 'unexpected_f',
              'am_p1_given2', 'am_p1_given2_simple', 
              'am_p2_given1', 'am_p2_given1_simple', 
              'conservative_log_ratio',
              'am_log_likelihood', 
            #   't_score',
            #   'mutual_information', 'am_odds_ratio_disc',
              'N', 'f1', 'f2', 'l1', 'l2']
FOCUS = adjust_assoc_columns(FOCUS_ORIG)
pd.set_option("display.float_format", '{:,.2f}'.format)
pd.set_option("display.max_colwidth", 80)
adv_am = seek_top_adv_am(DATE, ADV_F_FLOOR, TAG_TOP_STR, TAG_TOP_DIR)
adv_am

,key_SET,f_SET,dP1_SET,P1_SET,LRC_SET,G2_SET,MI_SET,odds_r_disc_SET,t_SET,N_SET,...,mean_t,mean_N,mean_f1,mean_f2,mean_expF,mean_unexpF,r_f_MIR,r_N_MIR,r_f1_MIR,r_f2_MIR
adv,,,,,,,,,,,,,,,,,,,,,
necessarily,NEGany~necessarily,42595,0.83,0.87,7.10,"230,257.34",1.30,2.17,196.05,72839589,...,110.48,37270759,1732696,"25,027.00","1,161.20","20,617.80",0.02,0.02,0.09,0.02
that,NEGany~that,164768,0.75,0.79,6.34,"831,137.25",1.26,1.94,383.56,72839589,...,217.42,37270759,1732696,"106,878.00","5,007.91","79,530.09",0.03,0.02,0.09,0.03
exactly,NEGany~exactly,43813,0.70,0.75,5.94,"210,126.60",1.23,1.82,197.11,72839589,...,109.68,37270759,1732696,"29,842.00","1,366.77","20,946.23",0.02,0.02,0.09,0.02
any,NEGany~any,15384,0.40,0.45,4.07,"50,880.96",1.01,1.25,111.95,72839589,...,69.16,37270759,1732696,"17,789.50",851.61,"7,373.39",0.07,0.02,0.09,0.03
remotely,NEGany~remotely,5661,0.30,0.34,3.40,"15,284.49",0.90,1.06,65.73,72839589,...,49.63,37270759,1732696,"9,383.50",558.48,"3,192.02",0.33,0.02,0.09,0.14
ever,NEGany~ever,5932,0.01,0.05,0.16,183.92,0.08,0.08,12.49,72839589,...,34.23,37270759,1732696,"59,567.50","2,918.83","2,401.67",0.79,0.02,0.09,0.04
yet,NEGany~yet,51867,0.50,0.54,4.65,"197,610.98",1.09,1.42,209.42,72839589,...,109.75,37270759,1732696,"48,289.00","2,156.07","23,937.43",0.01,0.02,0.09,0.01
immediately,NEGany~immediately,56099,0.54,0.58,4.86,"224,059.55",1.12,1.49,219.01,72839589,...,114.44,37270759,1732696,"49,084.00","2,215.00","26,036.00",0.01,0.02,0.09,0.01
particularly,NEGany~particularly,55527,0.07,0.11,1.38,"37,272.74",0.39,0.43,140.66,72839589,...,106.81,37270759,1732696,"263,335.50","12,304.83","20,080.17",0.17,0.02,0.09,0.03


In [56]:
NEG_HITS_PATH = HIT_TABLES_DIR /'RBdirect'/'ALL-RBdirect_final.parq'
POS_HITS_PATH = HIT_TABLES_DIR /'not-RBdirect'/f'{TAG}_not-RBdirect_final.parq'

In [57]:
adv_list = adv_am.index.to_list()
print_iter(adv_list, header=f'## Top {K} Most Negative Adverbs',bullet = '1.')


## Top 8 Most Negative Adverbs
1. necessarily
1. that
1. exactly
1. any
1. remotely
1. ever
1. yet
1. immediately
1. particularly
1. inherently
1. terribly


## Top 8 Most Negative Adverbs
1. necessarily
1. that
1. exactly
1. any
1. remotely
1. ever
1. yet
1. immediately
1. particularly
1. inherently
1. terribly



## Load AM table for `adv~adj` comparison (bigram composition)

In [58]:
blind_am_iter = AM_DF_DIR.joinpath('adv_adj').rglob(f'AdvAdj_{TAG}*min{BIGRAM_F_FLOOR}x_extra.parq')
blam_dict = {blamp.parent.parent.name.strip('ANY'): blamp for blamp in blind_am_iter}
print(pd.Series(blam_dict)
       .to_frame('path to "context-blind" AM scores')
       .to_markdown(tablefmt='rounded_grid', maxcolwidths=[None, 72]))


╭────────┬──────────────────────────────────────────────────────────────────────────╮
│        │ path to "context-blind" AM scores                                        │
├────────┼──────────────────────────────────────────────────────────────────────────┤
│ mirror │ /share/compling/projects/sanpi/results/assoc_df/adv_adj/ANYmirror/extra/ │
│        │ AdvAdj_ALL_any-mirror_final-freq_min50x_extra.parq                       │
├────────┼──────────────────────────────────────────────────────────────────────────┤
│ direct │ /share/compling/projects/sanpi/results/assoc_df/adv_adj/ANYdirect/extra/ │
│        │ AdvAdj_ALL_any-direct_final-freq_min50x_extra.parq                       │
╰────────┴──────────────────────────────────────────────────────────────────────────╯


In [59]:

def peek_am(peek_metric, blamin):
    peek = blamin.reset_index().groupby('l1').apply(
        lambda x: x.nlargest(1, peek_metric)
        ).reset_index(drop=True).set_index('key')
    print(f'\n_Bigrams with the highest `{peek_metric}` value for each adverb_\n')
    return peek.sort_values(peek_metric, ascending=False)

blind_priority_cols = METRIC_PRIORITY_DICT[f'{TAG}_blind']
blam_dfs = {}
for blam_kind, blam_path in blam_dict.items():
    print(f'\n### Loading `{blam_kind}` AM scores\n\n Path: `{blam_path.relative_to(AM_DF_DIR.parent)}`\n')
    blamin = pd.read_parquet(
        blam_path, engine='pyarrow',
        filters=[('l1', 'in', adv_list)],
        columns=FOCUS_DICT[TAG]['adv_adj'])
    blamin['dataset']=blam_kind
    blamin = catify(adjust_ams(blamin),
                    reverse=True)
    
    for peek_metric in blind_priority_cols:
        nb_show_table(peek_am(peek_metric, blamin).filter(blind_priority_cols), n_dec=3)
    blamin.index = f'[_{blam_kind}_] ' + blamin.index
    blam_dfs[blam_kind] = blamin


### Loading `mirror` AM scores

 Path: `assoc_df/adv_adj/ANYmirror/extra/AdvAdj_ALL_any-mirror_final-freq_min50x_extra.parq`


_Bigrams with the highest `LRC` value for each adverb_


|                              |   `LRC` |   `deltaP_mean` |   `deltaP_max` |   `unexp_r` |
|:-----------------------------|--------:|----------------:|---------------:|------------:|
| **exactly~alike**            |   7.793 |           0.147 |          0.210 |       0.997 |
| **immediately~recognizable** |   5.980 |           0.069 |          0.090 |       0.992 |
| **any~closer**               |   5.679 |           0.063 |          0.069 |       0.990 |
| **terribly~wrong**           |   4.859 |           0.222 |          0.363 |       0.967 |
| **inherently~wrong**         |   4.797 |           0.215 |          0.345 |       0.966 |
| **particularly~noteworthy**  |   4.227 |           0.107 |          0.207 |       0.964 |
| **ever~certain**             |   4.147 |           0.052 |          0.077 |  


### Loading `mirror` AM scores

 Path: `assoc_df/adv_adj/ANYmirror/extra/AdvAdj_ALL_any-mirror_final-freq_min50x_extra.parq`


_Bigrams with the highest `LRC` value for each adverb_


|                              |   `LRC` |   `deltaP_mean` |   `deltaP_max` |   `unexp_r` |
|:-----------------------------|--------:|----------------:|---------------:|------------:|
| **exactly~alike**            |   7.793 |           0.147 |          0.210 |       0.997 |
| **immediately~recognizable** |   5.980 |           0.069 |          0.090 |       0.992 |
| **any~closer**               |   5.679 |           0.063 |          0.069 |       0.990 |
| **terribly~wrong**           |   4.859 |           0.222 |          0.363 |       0.967 |
| **inherently~wrong**         |   4.797 |           0.215 |          0.345 |       0.966 |
| **particularly~noteworthy**  |   4.227 |           0.107 |          0.207 |       0.964 |
| **ever~certain**             |   4.147 |           0.052 |          0.077 |       0.963 |
| **that~great**               |   3.895 |           0.059 |          0.059 |       0.947 |
| **necessarily~wrong**        |   3.455 |           0.096 |          0.183 |       0.937 |
| **remotely~close**           |   3.339 |           0.062 |          0.106 |       0.929 |


_Bigrams with the highest `deltaP_mean` value for each adverb_


|                             |   `LRC` |   `deltaP_mean` |   `deltaP_max` |   `unexp_r` |
|:----------------------------|--------:|----------------:|---------------:|------------:|
| **terribly~wrong**          |   4.859 |           0.222 |          0.363 |       0.967 |
| **inherently~wrong**        |   4.797 |           0.215 |          0.345 |       0.966 |
| **any~better**              |   5.061 |           0.186 |          0.342 |       0.976 |
| **exactly~alike**           |   7.793 |           0.147 |          0.210 |       0.997 |
| **immediately~available**   |   4.802 |           0.125 |          0.224 |       0.974 |
| **particularly~noteworthy** |   4.227 |           0.107 |          0.207 |       0.964 |
| **necessarily~wrong**       |   3.455 |           0.096 |          0.183 |       0.937 |
| **remotely~close**          |   3.339 |           0.062 |          0.106 |       0.929 |
| **that~great**              |   3.895 |           0.059 |          0.059 |       0.947 |
| **ever~certain**            |   4.147 |           0.052 |          0.077 |       0.963 |


_Bigrams with the highest `deltaP_max` value for each adverb_


|                             |   `LRC` |   `deltaP_mean` |   `deltaP_max` |   `unexp_r` |
|:----------------------------|--------:|----------------:|---------------:|------------:|
| **terribly~wrong**          |   4.859 |           0.222 |          0.363 |       0.967 |
| **inherently~wrong**        |   4.797 |           0.215 |          0.345 |       0.966 |
| **any~better**              |   5.061 |           0.186 |          0.342 |       0.976 |
| **immediately~available**   |   4.802 |           0.125 |          0.224 |       0.974 |
| **exactly~alike**           |   7.793 |           0.147 |          0.210 |       0.997 |
| **particularly~noteworthy** |   4.227 |           0.107 |          0.207 |       0.964 |
| **necessarily~wrong**       |   3.455 |           0.096 |          0.183 |       0.937 |
| **remotely~close**          |   3.339 |           0.062 |          0.106 |       0.929 |
| **that~good**               |   2.295 |           0.055 |          0.094 |       0.834 |
| **ever~certain**            |   4.147 |           0.052 |          0.077 |       0.963 |


_Bigrams with the highest `unexp_r` value for each adverb_


|                              |   `LRC` |   `deltaP_mean` |   `deltaP_max` |   `unexp_r` |
|:-----------------------------|--------:|----------------:|---------------:|------------:|
| **exactly~alike**            |   7.793 |           0.147 |          0.210 |       0.997 |
| **immediately~recognizable** |   5.980 |           0.069 |          0.090 |       0.992 |
| **any~closer**               |   5.679 |           0.063 |          0.069 |       0.990 |
| **terribly~amiss**           |   3.986 |           0.050 |          0.089 |       0.970 |
| **inherently~wrong**         |   4.797 |           0.215 |          0.345 |       0.966 |
| **particularly~noteworthy**  |   4.227 |           0.107 |          0.207 |       0.964 |
| **ever~certain**             |   4.147 |           0.052 |          0.077 |       0.963 |
| **that~great**               |   3.895 |           0.059 |          0.059 |       0.947 |
| **necessarily~wrong**        |   3.455 |           0.096 |          0.183 |       0.937 |
| **remotely~related**         |   2.970 |           0.024 |          0.029 |       0.935 |


### Loading `direct` AM scores

 Path: `assoc_df/adv_adj/ANYdirect/extra/AdvAdj_ALL_any-direct_final-freq_min50x_extra.parq`


_Bigrams with the highest `LRC` value for each adverb_


|                             |   `LRC` |   `deltaP_mean` |   `deltaP_max` |   `unexp_r` |
|:----------------------------|--------:|----------------:|---------------:|------------:|
| **remotely~detonated**      |  12.691 |           0.440 |          0.876 |       1.000 |
| **yet~unborn**              |  10.135 |           0.360 |          0.715 |       0.998 |
| **inherently~governmental** |   8.919 |           0.169 |          0.332 |       0.998 |
| **immediately~accretive**   |   8.539 |           0.226 |          0.450 |       0.997 |
| **exactly~alike**           |   8.461 |           0.145 |          0.243 |       0.997 |
| **that~purported**          |   8.416 |           0.391 |          0.782 |       0.996 |
| **ever~olympic**            |   8.231 |           0.222 |          0.442 |       0.996 |
| **necessarily~indicative**  |   8.025 |           0.100 |          0.171 |       0.996 |
| **terribly~awry**           |   8.019 |           0.131 |          0.259 |       0.997 |
| **any~happier**             |   6.748 |           0.043 |          0.058 |       0.992 |
| **particularly~hard-hit**   |   5.642 |           0.194 |          0.387 |       0.982 |


_Bigrams with the highest `deltaP_mean` value for each adverb_


|                             |   `LRC` |   `deltaP_mean` |   `deltaP_max` |   `unexp_r` |
|:----------------------------|--------:|----------------:|---------------:|------------:|
| **remotely~detonated**      |  12.691 |           0.440 |          0.876 |       1.000 |
| **that~purported**          |   8.416 |           0.391 |          0.782 |       0.996 |
| **yet~unborn**              |  10.135 |           0.360 |          0.715 |       0.998 |
| **immediately~appealable**  |   8.409 |           0.273 |          0.545 |       0.998 |
| **ever~quarterly**          |   8.049 |           0.224 |          0.446 |       0.997 |
| **particularly~hard-hit**   |   5.642 |           0.194 |          0.387 |       0.982 |
| **any~better**              |   5.227 |           0.171 |          0.325 |       0.974 |
| **inherently~governmental** |   8.919 |           0.169 |          0.332 |       0.998 |
| **exactly~alike**           |   8.461 |           0.145 |          0.243 |       0.997 |
| **terribly~awry**           |   8.019 |           0.131 |          0.259 |       0.997 |
| **necessarily~indicative**  |   8.025 |           0.100 |          0.171 |       0.996 |


_Bigrams with the highest `deltaP_max` value for each adverb_


|                             |   `LRC` |   `deltaP_mean` |   `deltaP_max` |   `unexp_r` |
|:----------------------------|--------:|----------------:|---------------:|------------:|
| **remotely~detonated**      |  12.691 |           0.440 |          0.876 |       1.000 |
| **that~purported**          |   8.416 |           0.391 |          0.782 |       0.996 |
| **yet~unborn**              |  10.135 |           0.360 |          0.715 |       0.998 |
| **immediately~appealable**  |   8.409 |           0.273 |          0.545 |       0.998 |
| **ever~quarterly**          |   8.049 |           0.224 |          0.446 |       0.997 |
| **particularly~hard-hit**   |   5.642 |           0.194 |          0.387 |       0.982 |
| **inherently~governmental** |   8.919 |           0.169 |          0.332 |       0.998 |
| **any~better**              |   5.227 |           0.171 |          0.325 |       0.974 |
| **terribly~awry**           |   8.019 |           0.131 |          0.259 |       0.997 |
| **exactly~alike**           |   8.461 |           0.145 |          0.243 |       0.997 |
| **necessarily~indicative**  |   8.025 |           0.100 |          0.171 |       0.996 |


_Bigrams with the highest `unexp_r` value for each adverb_


|                             |   `LRC` |   `deltaP_mean` |   `deltaP_max` |   `unexp_r` |
|:----------------------------|--------:|----------------:|---------------:|------------:|
| **remotely~detonated**      |  12.691 |           0.440 |          0.876 |       1.000 |
| **yet~unborn**              |  10.135 |           0.360 |          0.715 |       0.998 |
| **inherently~governmental** |   8.919 |           0.169 |          0.332 |       0.998 |
| **immediately~appealable**  |   8.409 |           0.273 |          0.545 |       0.998 |
| **terribly~awry**           |   8.019 |           0.131 |          0.259 |       0.997 |
| **exactly~alike**           |   8.461 |           0.145 |          0.243 |       0.997 |
| **ever~quarterly**          |   8.049 |           0.224 |          0.446 |       0.997 |
| **that~purported**          |   8.416 |           0.391 |          0.782 |       0.996 |
| **necessarily~indicative**  |   8.025 |           0.100 |          0.171 |       0.996 |
| **any~happier**             |   6.748 |           0.043 |          0.058 |       0.992 |
| **particularly~hard-hit**   |   5.642 |           0.194 |          0.387 |       0.982 |



In [60]:

blam_df = pd.concat(blam_dfs.values()).sort_values(blind_priority_cols[0], ascending=False)
# adv_adj_am = adv_adj_am.loc[adv_adj_am.l1.isin(adv_list), :]
print(f'### Top 15 *context-blind* `{blind_priority_cols[0]}` values across all adverbs and datasets\n')
nb_show_table(blam_df.head(15))

### Top 15 *context-blind* `LRC` values across all adverbs and datasets


|                                        |   `f` |   `dP1` |   `LRC` |   `P1` |      `G2` | `l1`        | `l2`         |    `f1` |   `f2` |        `N` |   `exp_f` |   `unexp_f` |   `unexp_r` |   `odds_r_disc` |   `t` |   `MI` |   `dP2` |   `P2` |   `deltaP_max` |   `deltaP_mean` | `dataset`   |
|:---------------------------------------|------:|--------:|--------:|-------:|----------:|:------------|:-------------|--------:|-------:|-----------:|----------:|------------:|------------:|----------------:|------:|-------:|--------:|-------:|---------------:|----------------:|:------------|
| **[_direct_] remotely~detonated**      |    78 |    0.88 |   12.69 |   0.88 |  1,243.75 | remotely    | detonated    |  16,426 |     89 | 72,839,589 |      0.02 |       77.98 |        1.00 |            4.48 |  8.83 |   3.59 |    0.00 |   0.00 |           0.88 |            0.44 | direct      |
| **[_direct_] yet~unborn**           

### Top 15 *context-blind* `LRC` values across all adverbs and datasets


|                                        |   `f` |   `dP1` |   `LRC` |   `P1` |      `G2` | `l1`        | `l2`         |    `f1` |   `f2` |        `N` |   `exp_f` |   `unexp_f` |   `unexp_r` |   `odds_r_disc` |   `t` |   `MI` |   `dP2` |   `P2` |   `deltaP_max` |   `deltaP_mean` | `dataset`   |
|:---------------------------------------|------:|--------:|--------:|-------:|----------:|:------------|:-------------|--------:|-------:|-----------:|----------:|------------:|------------:|----------------:|------:|-------:|--------:|-------:|---------------:|----------------:|:------------|
| **[_direct_] remotely~detonated**      |    78 |    0.88 |   12.69 |   0.88 |  1,243.75 | remotely    | detonated    |  16,426 |     89 | 72,839,589 |      0.02 |       77.98 |        1.00 |            4.48 |  8.83 |   3.59 |    0.00 |   0.00 |           0.88 |            0.44 | direct      |
| **[_direct_] yet~unborn**              |   372 |    0.72 |   10.14 |   0.72 |  4,319.00 | yet         | unborn       |  95,763 |    519 | 72,839,589 |      0.68 |      371.32 |        1.00 |            3.28 | 19.25 |   2.74 |    0.00 |   0.00 |           0.72 |            0.36 | direct      |
| **[_direct_] inherently~governmental** |   253 |    0.33 |    8.92 |   0.33 |  2,742.59 | inherently  | governmental |  47,803 |    761 | 72,839,589 |      0.50 |      252.50 |        1.00 |            2.88 | 15.87 |   2.70 |    0.01 |   0.01 |           0.33 |            0.17 | direct      |
| **[_direct_] remotely~exploitable**    |   145 |    0.15 |    8.79 |   0.15 |  1,613.38 | remotely    | exploitable  |  16,426 |    986 | 72,839,589 |      0.22 |      144.78 |        1.00 |            2.89 | 12.02 |   2.81 |    0.01 |   0.01 |           0.15 |            0.08 | direct      |
| **[_direct_] immediately~accretive**   |   236 |    0.45 |    8.54 |   0.45 |  2,406.67 | immediately | accretive    |  96,973 |    523 | 72,839,589 |      0.70 |      235.30 |        1.00 |            2.79 | 15.32 |   2.53 |    0.00 |   0.00 |           0.45 |            0.23 | direct      |
| **[_direct_] exactly~alike**           | 2,768 |    0.24 |    8.46 |   0.24 | 26,963.38 | exactly     | alike        |  58,643 | 11,375 | 72,839,589 |      9.16 |    2,758.84 |        1.00 |            2.62 | 52.44 |   2.48 |    0.05 |   0.05 |           0.24 |            0.14 | direct      |
| **[_direct_] that~purported**          |    73 |    0.78 |    8.42 |   0.78 |    758.47 | that        | purported    | 208,262 |     93 | 72,839,589 |      0.27 |       72.73 |        1.00 |            3.10 |  8.51 |   2.44 |    0.00 |   0.00 |           0.78 |            0.39 | direct      |
| **[_direct_] immediately~appealable**  |    76 |    0.55 |    8.41 |   0.55 |    815.23 | immediately | appealable   |  96,973 |    139 | 72,839,589 |      0.19 |       75.81 |        1.00 |            2.96 |  8.70 |   2.61 |    0.00 |   0.00 |           0.55 |            0.27 | direct      |
| **[_direct_] immediately~adjacent**    | 1,595 |    0.33 |    8.31 |   0.34 | 15,089.64 | immediately | adjacent     |  96,973 |  4,756 | 72,839,589 |      6.33 |    1,588.67 |        1.00 |            2.59 | 39.78 |   2.40 |    0.02 |   0.02 |           0.33 |            0.18 | direct      |
| **[_direct_] yet~unnamed**             |   736 |    0.35 |    8.29 |   0.35 |  7,048.15 | yet         | unnamed      |  95,763 |  2,107 | 72,839,589 |      2.77 |      733.23 |        1.00 |            2.61 | 27.03 |   2.42 |    0.01 |   0.01 |           0.35 |            0.18 | direct      |
| **[_direct_] ever~olympic**            |   218 |    0.44 |    8.23 |   0.44 |  2,141.80 | ever        | olympic      | 114,075 |    492 | 72,839,589 |      0.77 |      217.23 |        1.00 |            2.71 | 14.71 |   2.45 |    0.00 |   0.00 |           0.44 |            0.22 | direct      |
| **[_direct_] ever~quarterly**          |   137 |    0.45 |    8.05 |   0.45 |  1,349.65 | ever        | quarterly    | 114,075 |    306 | 72,839,589 |      0.48 |      136.52 |        1.00 |            2.71 | 11.66 |   2.46 |    0.00 |   0.00 |           0.45 |            0.22 | direct      |
| **[_direct_] necessarily~indicative**  | 1,400 |    0.17 |    8.03 |   0.17 | 13,028.00 | necessarily | indicative   |  48,947 |  8,148 | 72,839,589 |      5.48 |    1,394.52 |        1.00 |            2.50 | 37.27 |   2.41 |    0.03 |   0.03 |           0.17 |            0.10 | direct      |
| **[_direct_] terribly~awry**           |   180 |    0.26 |    8.02 |   0.26 |  1,770.97 | terribly    | awry         |  58,964 |    692 | 72,839,589 |      0.56 |      179.44 |        1.00 |            2.64 | 13.37 |   2.51 |    0.00 |   0.00 |           0.26 |            0.13 | direct      |
| **[_direct_] yet~unspecified**         |   204 |    0.33 |    7.86 |   0.34 |  1,933.20 | yet         | unspecified  |  95,763 |    607 | 72,839,589 |      0.80 |      203.20 |        1.00 |            2.59 | 14.23 |   2.41 |    0.00 |   0.00 |           0.33 |            0.17 | direct      |



In [61]:
blam_df.filter(like='mirror', axis=0)

,f,dP1,LRC,P1,G2,l1,l2,f1,f2,N,...,unexp_f,unexp_r,odds_r_disc,t,MI,dP2,P2,deltaP_max,deltaP_mean,dataset
key,,,,,,,,,,,,,,,,,,,,,
[_mirror_] exactly~alike,88,0.21,7.79,0.21,880.53,exactly,alike,1041,417,1701929,...,87.74,1.00,2.68,9.35,2.54,0.08,0.08,0.21,0.15,mirror
[_mirror_] immediately~recognizable,58,0.09,5.98,0.09,456.86,immediately,recognizable,1195,640,1701929,...,57.55,0.99,2.18,7.56,2.11,0.05,0.05,0.09,0.07,mirror
[_mirror_] any~closer,69,0.07,5.68,0.07,506.03,any,closer,1197,993,1701929,...,68.30,0.99,2.05,8.22,1.99,0.06,0.06,0.07,0.06,mirror
[_mirror_] any~better,419,0.03,5.06,0.03,"2,497.00",any,better,1197,14013,1701929,...,409.14,0.98,1.83,19.99,1.63,0.34,0.35,0.34,0.19,mirror
[_mirror_] terribly~wrong,1726,0.08,4.86,0.08,"9,304.94",terribly,wrong,4610,20880,1701929,...,"1,669.44",0.97,1.72,40.18,1.48,0.36,0.37,0.36,0.22,mirror
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
[_mirror_] ever~available,51,0.00,0.00,0.00,11.46,ever,available,5060,10284,1701929,...,20.42,0.40,0.23,2.86,0.22,0.00,0.01,0.00,0.00,mirror
[_mirror_] terribly~important,79,-0.00,0.00,0.00,-15.18,terribly,important,4610,43671,1701929,...,-39.29,-0.50,-0.18,-4.42,-0.18,-0.01,0.02,-0.00,-0.00,mirror
[_mirror_] that~different,78,-0.00,0.00,0.00,-14.90,that,different,5494,36166,1701929,...,-38.75,-0.50,-0.18,-4.39,-0.18,-0.01,0.01,-0.00,-0.00,mirror


In [66]:



hits_df = load_hit_table(
    adv_list, pos_hits=POS_HITS_PATH, neg_hits=NEG_HITS_PATH)
hits_df

Saving as parquet
  partitioned by `['adv_form_lower']`...

| adv_form_lower   |   count |
|:-----------------|--------:|
| particularly     | 101,341 |
| that             |  53,554 |
| yet              |  22,946 |
| immediately      |  22,486 |
| ever             |  21,513 |
| exactly          |  15,047 |
| necessarily      |  12,891 |
| terribly         |  12,762 |
| inherently       |   9,628 |
| any              |   7,948 |
| remotely         |   3,656 |

> no more than 8,600 rows per individual `group-[#].parquet`
  - max rows in writing batch = 4,301
  - min rows in writing batch = 1,434

✓ Sample of bigram tokens for `ALL-Top8[5000]`  successfully saved as  
> "/share/compling/projects/sanpi/results/top_AM/ALL/ALL-Top8/ALL-Top8adv_sample-hits_2024-07-29.parq"
* Total time to write partitioned parquet ⇾  `00:00:00.951`


,text_window,token_str,adv_form_lower,adj_form_lower,bigram_lower,polarity
hit_id,,,,,,
apw_eng_19970410_0298_7:6-7,no margin of error was immediately available .,no margin of error was immediately available .,immediately,available,immediately_available,pos
apw_eng_19970413_0437_16:14-15,it on the way to its first ever championship final appearance .,"the Swiss won 2-1 , helping it on the way to its first ever Championship fin...",ever,championship,ever_championship,pos
apw_eng_19970413_0812_24:5-6,trainer shug mcgaughey was particularly proud of accelerator after four cons...,Trainer Shug McGaughey was particularly proud of Accelerator after four cons...,particularly,proud,particularly_proud,pos
apw_eng_19970414_0374_3:5-6,`` this project is particularly important because of the economic conditions in,`` This project is particularly important because of the economic conditions...,particularly,important,particularly_important,pos
apw_eng_19970414_0713_4:5-6,no other information was immediately available on the meeting between the two,no other information was immediately available on the meeting between the tw...,immediately,available,immediately_available,pos
...,...,...,...,...,...,...
pcc_eng_val_3.09057_x49184_235:13-15-16,"times , but he 's not even that big of a threat to score the","He can be a maestro offensively at times , but he 's not even that big of a ...",that,big,that_big,neg
pcc_eng_val_3.10127_x50910_037:13-15-16,questions about things i 'm not even that interested in and i 'm somebody with,Then I get asked a lot of questions about things I 'm not even that interest...,that,interested,that_interested,neg
pcc_eng_val_3.10936_x52202_02:20-21-22,and moving for just 3 is n't exactly healthy or in your best interest as,"Sitting for over 13 hours a day , sleeping for 8 more , and moving for just ...",exactly,healthy,exactly_healthy,neg


Saving as parquet
  partitioned by `['adv_form_lower']`...

| adv_form_lower   |   count |
|:-----------------|--------:|
| particularly     | 101,341 |
| that             |  53,554 |
| yet              |  22,946 |
| immediately      |  22,486 |
| ever             |  21,513 |
| exactly          |  15,047 |
| necessarily      |  12,891 |
| terribly         |  12,762 |
| inherently       |   9,628 |
| any              |   7,948 |
| remotely         |   3,656 |

> no more than 8,600 rows per individual `group-[#].parquet`
  - max rows in writing batch = 4,301
  - min rows in writing batch = 1,434

✓ Sample of bigram tokens for `ALL-Top8[5000]`  successfully saved as  
> "/share/compling/projects/sanpi/results/top_AM/ALL/ALL-Top8/ALL-Top8adv_sample-hits_2024-07-29.parq"
* Total time to write partitioned parquet ⇾  `00:00:00.113`


In [63]:
# nb_show_table(hits_df.sort_index(axis=1).sample(13).iloc[:, :4])

```python
nb_show_table(hits_df.sort_index(axis=1).sample(13).iloc[:, :4])
```

|                                               | `adj_form_lower` | `adv_form_lower` | `bigrlower`             | `text_window`                                                                                     |
|:----------------------------------------------|:-----------------|:-----------------|:------------------------|:--------------------------------------------------------------------------------------------------|
| **pcc_eng_26_068.6567_x1093885_3:5-6**        | popular          | ever             | ever_popular            | spacious mid terrace in ever popular location                                                     |
| **pcc_eng_28_027.3336_x0425352_21:4-5-6**     | hea              | necessarily      | necessarily_hea         | the end is not necessarily hea , but it 's a solid                                                |
| **nyt_eng_19950702_0006_49:8-9**              | ugly             | particularly     | particularly_ugly       | he 'll make a face after a particularly ugly swing .                                              |
| **pcc_eng_24_010.6893_x0156421_04:12-13**     | effective        | particularly     | particularly_effective  | analog to album pre-sales , and are particularly effective for streaming fans who may have        |
| **nyt_eng_19960708_0547_13:26-27-28**         | addictive        | necessarily      | necessarily_addictive   | and -rrb- `` says cigarettes are n't necessarily addictive ; some say milk is bad                 |
| **nyt_eng_20040127_0066_8:25-26**             | disturbing       | particularly     | particularly_disturbing | police department employees , but it is particularly disturbing when it involves a scientist who  |
| **pcc_eng_25_007.0915_x0098850_12:14-15-16**  | useful           | particularly     | particularly_useful     | value and blog traffic but are not particularly useful for true leaders , who invent              |
| **pcc_eng_14_041.1082_x0648041_080:08-09-10** | unique           | exactly          | exactly_unique          | award - winning development project is not exactly unique .                                       |
| **pcc_eng_00_050.2537_x0795891_224:10-11**    | concerned        | particularly     | particularly_concerned  | light of climate change , i am particularly concerned about the power that corporations have      |
| **pcc_eng_22_077.2483_x1232367_62:4-5**       | true             | particularly     | particularly_true       | that could prove particularly true for " black ops . "                                            |
| **nyt_eng_20060308_0284_27:4-5-6**            | different        | that             | that_different          | `` it 's not that different from playing shortstop . ''                                           |
| **nyt_eng_20070425_0212_10:1-2-3**            | wrong            | necessarily      | necessarily_wrong       | nothing necessarily wrong with that .                                                             |
| **pcc_eng_07_027.5704_x0429654_14:27-28**     | vulnerable       | particularly     | particularly_vulnerable | to construct sewers and reduce flooding in particularly vulnerable areas in the borough of queens |



## Set adverb and collect specific values

In [64]:
adv_am

,key_SET,f_SET,dP1_SET,P1_SET,LRC_SET,G2_SET,MI_SET,odds_r_disc_SET,t_SET,N_SET,...,mean_t,mean_N,mean_f1,mean_f2,mean_expF,mean_unexpF,r_f_MIR,r_N_MIR,r_f1_MIR,r_f2_MIR
adv,,,,,,,,,,,,,,,,,,,,,
necessarily,NEGany~necessarily,42595,0.83,0.87,7.10,"230,257.34",1.30,2.17,196.05,72839589,...,110.48,37270759,1732696,"25,027.00","1,161.20","20,617.80",0.02,0.02,0.09,0.02
that,NEGany~that,164768,0.75,0.79,6.34,"831,137.25",1.26,1.94,383.56,72839589,...,217.42,37270759,1732696,"106,878.00","5,007.91","79,530.09",0.03,0.02,0.09,0.03
exactly,NEGany~exactly,43813,0.70,0.75,5.94,"210,126.60",1.23,1.82,197.11,72839589,...,109.68,37270759,1732696,"29,842.00","1,366.77","20,946.23",0.02,0.02,0.09,0.02
any,NEGany~any,15384,0.40,0.45,4.07,"50,880.96",1.01,1.25,111.95,72839589,...,69.16,37270759,1732696,"17,789.50",851.61,"7,373.39",0.07,0.02,0.09,0.03
remotely,NEGany~remotely,5661,0.30,0.34,3.40,"15,284.49",0.90,1.06,65.73,72839589,...,49.63,37270759,1732696,"9,383.50",558.48,"3,192.02",0.33,0.02,0.09,0.14
ever,NEGany~ever,5932,0.01,0.05,0.16,183.92,0.08,0.08,12.49,72839589,...,34.23,37270759,1732696,"59,567.50","2,918.83","2,401.67",0.79,0.02,0.09,0.04
yet,NEGany~yet,51867,0.50,0.54,4.65,"197,610.98",1.09,1.42,209.42,72839589,...,109.75,37270759,1732696,"48,289.00","2,156.07","23,937.43",0.01,0.02,0.09,0.01
immediately,NEGany~immediately,56099,0.54,0.58,4.86,"224,059.55",1.12,1.49,219.01,72839589,...,114.44,37270759,1732696,"49,084.00","2,215.00","26,036.00",0.01,0.02,0.09,0.01
particularly,NEGany~particularly,55527,0.07,0.11,1.38,"37,272.74",0.39,0.43,140.66,72839589,...,106.81,37270759,1732696,"263,335.50","12,304.83","20,080.17",0.17,0.02,0.09,0.03


In [65]:
adv_adj_am

NameError: name 'adv_adj_am' is not defined

In [ ]:
for adverb in adv_am.index:
    sample_adv_bigrams(
        adverb, data_tag=TAG, verbose=True,
        amdf=adv_adj_am, hits_df=hits_df,
        n_top_bigrams=BK, bigram_floor=BIGRAM_F_FLOOR)


## *necessarily*


|                                |   `f` |   `exp_f` |   `unexp_f` |   `dP1` |   `P1` |   `dP2` |   `P2` |   `LRC` |      `G2` |        `N` |   `f1` |    `f2` | `l1`        | `l2`           |
|:-------------------------------|------:|----------:|------------:|--------:|-------:|--------:|-------:|--------:|----------:|-----------:|-------:|--------:|:------------|:---------------|
| **necessarily~indicative**     | 1,400 |      5.48 |    1,394.52 |    0.17 |   0.17 |    0.03 |   0.03 |    8.03 | 13,028.00 | 72,839,589 | 48,947 |   8,148 | necessarily | indicative     |
| **necessarily~cause**          |    52 |      0.50 |       51.50 |    0.07 |   0.07 |    0.00 |   0.00 |    5.46 |    383.88 | 72,839,589 | 48,947 |     743 | necessarily | cause          |
| **necessarily~representative** |   492 |     12.33 |      479.67 |    0.03 |   0.03 |    0.01 |   0.01 |    4.97 |  2,685.16 | 72,839,589 | 48,947 |  18,355 | necessarily | representative |
| **necessarily~pred

KeyboardInterrupt: 

```python
for adverb in adv_am.index: 
    sample_adv_bigrams(adverb, adv_adj_am, hits_df)
```

## *necessarily*


|                                |   `f` |   `exp_f` |   `unexp_f` |   `dP1` |   `P1` |   `dP2` |   `P2` |   `LRC` |      `G2` |        `N` |   `f1` |    `f2` | `l1`        | `l2`           |
|:-------------------------------|------:|----------:|------------:|--------:|-------:|--------:|-------:|--------:|----------:|-----------:|-------:|--------:|:------------|:---------------|
| **necessarily~indicative**     | 1,400 |      5.48 |    1,394.52 |    0.17 |   0.17 |    0.03 |   0.03 |    8.03 | 13,028.00 | 72,839,589 | 48,947 |   8,148 | necessarily | indicative     |
| **necessarily~cause**          |    52 |      0.50 |       51.50 |    0.07 |   0.07 |    0.00 |   0.00 |    5.46 |    383.88 | 72,839,589 | 48,947 |     743 | necessarily | cause          |
| **necessarily~representative** |   492 |     12.33 |      479.67 |    0.03 |   0.03 |    0.01 |   0.01 |    4.97 |  2,685.16 | 72,839,589 | 48,947 |  18,355 | necessarily | representative |
| **necessarily~true**           | 3,460 |    155.66 |    3,304.34 |    0.01 |   0.01 |    0.07 |   0.07 |    4.35 | 15,129.32 | 72,839,589 | 48,947 | 231,639 | necessarily | true           |
| **necessarily~synonymous**     |   167 |      5.54 |      161.46 |    0.02 |   0.02 |    0.00 |   0.00 |    4.25 |    818.37 | 72,839,589 | 48,947 |   8,245 | necessarily | synonymous     |
| **necessarily~incompatible**   |   112 |      3.58 |      108.42 |    0.02 |   0.02 |    0.00 |   0.00 |    4.14 |    556.70 | 72,839,589 | 48,947 |   5,332 | necessarily | incompatible   |
| **necessarily~reflective**     |   183 |      7.55 |      175.45 |    0.02 |   0.02 |    0.00 |   0.00 |    3.97 |    819.22 | 72,839,589 | 48,947 |  11,237 | necessarily | reflective     |
| **necessarily~predictive**     |    58 |      1.63 |       56.37 |    0.02 |   0.02 |    0.00 |   0.00 |    3.95 |    303.20 | 72,839,589 | 48,947 |   2,421 | necessarily | predictive     |
| **necessarily~incomplete**     |   124 |      5.13 |      118.87 |    0.02 |   0.02 |    0.00 |   0.00 |    3.81 |    554.34 | 72,839,589 | 48,947 |   7,634 | necessarily | incomplete     |
| **necessarily~evil**           |   224 |     15.26 |      208.74 |    0.01 |   0.01 |    0.00 |   0.00 |    3.30 |    788.90 | 72,839,589 | 48,947 |  22,706 | necessarily | evil           |


1. necessarily_indicative
   >  While these lows do n't occur like clockwork every four years , and past performance is not necessarily indicative of future results , the historical record is rather impressive .

2. necessarily_cause
   >  This is not necessarily cause for alarm .

3. necessarily_representative
   >  Let 's just be clear : what some " Mormon Apologists " write is not necessarily representative of what most " Mormons " think or believe .

4. necessarily_true
   >  " I must finally conclude that this proposition , I am , I exist , is necessarily true whenever it is put forward by me or conceived in my mind . " -

5. necessarily_synonymous
   >  And this discussion always seems to come back to what we find attractive and appealing , instead of what we know to be healthy ( which research shows is not necessarily synonymous with thin ) .

6. necessarily_incompatible
   >  The philosophies of habits and intensity are n't necessarily incompatible .

7. necessarily_reflective
   >  After reading a novel by Adichie , I feel more educated and more aware of the fact that my way of viewing the world is incomplete and not necessarily reflective of everyone 's experiences .

8. necessarily_predictive
   >  There are varying degrees of offensive prowess sprinkled throughout the list of the hardest-hitters in baseball , so it is n't necessarily predictive of anything other than solid-average offense - but it 's a good sign nevertheless .

9. necessarily_incomplete
   >  The TPC 's analysis is necessarily incomplete ; it considers only the rate reductions Romney has made public and not the tax preferences that he has said he will eliminate but has n't yet identified .

10. necessarily_evil
   >  Vampires are n't necessarily evil .

Saving Samples in /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/necessarily/...

Samples saved as...
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/necessarily/necessarily_indicative_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/necessarily/necessarily_cause_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/necessarily/necessarily_representative_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/necessarily/necessarily_true_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/necessarily/necessarily_synonymous_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/necessarily/necessarily_incompatible_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/necessarily/necessarily_reflective_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/necessarily/necessarily_predictive_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/necessarily/necessarily_incomplete_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/necessarily/necessarily_evil_50ex.csv

## *that*


|                     |    `f` |   `exp_f` |   `unexp_f` |   `dP1` |   `P1` |   `dP2` |   `P2` |   `LRC` |      `G2` |        `N` |    `f1` |    `f2` | `l1`   | `l2`       |
|:--------------------|-------:|----------:|------------:|--------:|-------:|--------:|-------:|--------:|----------:|-----------:|--------:|--------:|:-------|:-----------|
| **that~purported**  |     73 |      0.27 |       72.73 |    0.78 |   0.78 |    0.00 |   0.00 |    8.42 |    758.47 | 72,839,589 | 208,262 |      93 | that   | purported  |
| **that~uncommon**   |    804 |     32.34 |      771.66 |    0.07 |   0.07 |    0.00 |   0.00 |    4.43 |  3,680.42 | 72,839,589 | 208,262 |  11,312 | that   | uncommon   |
| **that~dissimilar** |    307 |     13.17 |      293.83 |    0.06 |   0.07 |    0.00 |   0.00 |    4.13 |  1,365.56 | 72,839,589 | 208,262 |   4,605 | that   | dissimilar |
| **that~bad**        | 19,708 |  1,228.13 |   18,479.87 |    0.04 |   0.05 |    0.09 |   0.09 |    4.01 | 74,955.43 | 72,839,589 | 208,262 | 429,537 | that   | bad        |
| **that~farfetched** |     93 |      3.79 |       89.21 |    0.07 |   0.07 |    0.00 |   0.00 |    3.75 |    423.24 | 72,839,589 | 208,262 |   1,324 | that   | farfetched |
| **that~great**      | 11,781 |    884.23 |   10,896.77 |    0.04 |   0.04 |    0.05 |   0.06 |    3.71 | 40,195.15 | 72,839,589 | 208,262 | 309,258 | that   | great      |
| **that~hard**       | 10,380 |    996.32 |    9,383.68 |    0.03 |   0.03 |    0.05 |   0.05 |    3.34 | 30,573.43 | 72,839,589 | 208,262 | 348,463 | that   | hard       |
| **that~stupid**     |  1,437 |    145.02 |    1,291.98 |    0.03 |   0.03 |    0.01 |   0.01 |    3.12 |  4,048.67 | 72,839,589 | 208,262 |  50,722 | that   | stupid     |
| **that~big**        |  7,073 |    865.10 |    6,207.90 |    0.02 |   0.02 |    0.03 |   0.03 |    2.96 | 17,624.62 | 72,839,589 | 208,262 | 302,567 | that   | big        |
| **that~easy**       | 12,825 |  1,657.83 |   11,167.17 |    0.02 |   0.02 |    0.05 |   0.06 |    2.91 | 30,976.21 | 72,839,589 | 208,262 | 579,827 | that   | easy       |


1. that_purported
   >  Benham noted that in Davis ' most recent appeals , his lawyers put into evidence affidavits that purported to show that executions by electrocution have been plagued by `` shocking and grotesque errors '' and that there is a substantial risk they result in `` unnecessary infliction of pain and disfigurement . ''

2. that_uncommon
   >  Perhaps such a practice was not that uncommon in Japan , especially during the Middle Ages .

3. that_dissimilar
   >  Many of the prisoners she met were " not that dissimilar from entrepreneurs , " she tells Chu .

4. that_bad
   >  The preview I watched was n't that bad although I was left a bit confused by the end with the development of the characters .

5. that_farfetched
   >  However , it makes me wonder how much of this ' farfetched ' film is really that farfetched .

6. that_great
   >  Is hard to believe that the support was not that great , but we did not give up , and we raised enough money to BUY and ultrasound .

7. that_hard
   >  I mean really , it 's not that hard to soak a pound of dry beans , and I love beans .

8. that_stupid
   >  I 'm not that stupid , seeing as obviously he has a thing for you ... " she said , smirking as a slight flush went over Kasumi 's cheeks .

9. that_big
   >  Kemp's Ridley turtles get up to 100 feet and 2 feet long which is not that big for a turtle .

10. that_easy
   >  However , it is not that easy to find the suitable terms to characterize a wine .

Saving Samples in /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/that/...

Samples saved as...
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/that/that_purported_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/that/that_uncommon_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/that/that_dissimilar_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/that/that_bad_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/that/that_farfetched_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/that/that_great_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/that/that_hard_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/that/that_stupid_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/that/that_big_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/that/that_easy_50ex.csv

## *exactly*


|                           |   `f` |   `exp_f` |   `unexp_f` |   `dP1` |   `P1` |   `dP2` |   `P2` |   `LRC` |      `G2` |        `N` |   `f1` |    `f2` | `l1`    | `l2`          |
|:--------------------------|------:|----------:|------------:|--------:|-------:|--------:|-------:|--------:|----------:|-----------:|-------:|--------:|:--------|:--------------|
| **exactly~alike**         | 2,768 |      9.16 |    2,758.84 |    0.24 |   0.24 |    0.05 |   0.05 |    8.46 | 26,963.38 | 72,839,589 | 58,643 |  11,375 | exactly | alike         |
| **exactly~opposite**      |   464 |      6.84 |      457.16 |    0.05 |   0.05 |    0.01 |   0.01 |    5.76 |  3,028.33 | 72,839,589 | 58,643 |   8,491 | exactly | opposite      |
| **exactly~right**         | 6,323 |    115.21 |    6,207.79 |    0.04 |   0.04 |    0.11 |   0.11 |    5.74 | 39,191.65 | 72,839,589 | 58,643 | 143,095 | exactly | right         |
| **exactly~sure**          | 9,157 |    211.60 |    8,945.40 |    0.03 |   0.03 |    0.15 |   0.16 |    5.40 | 52,863.19 | 72,839,589 | 58,643 | 262,825 | exactly | sure          |
| **exactly~zero**          |   305 |      7.65 |      297.35 |    0.03 |   0.03 |    0.01 |   0.01 |    4.86 |  1,664.57 | 72,839,589 | 58,643 |   9,500 | exactly | zero          |
| **exactly~parallel**      |   205 |      5.22 |      199.78 |    0.03 |   0.03 |    0.00 |   0.00 |    4.72 |  1,111.99 | 72,839,589 | 58,643 |   6,488 | exactly | parallel      |
| **exactly~stellar**       |   172 |      4.33 |      167.67 |    0.03 |   0.03 |    0.00 |   0.00 |    4.68 |    936.96 | 72,839,589 | 58,643 |   5,379 | exactly | stellar       |
| **exactly~analogous**     |   103 |      2.47 |      100.53 |    0.03 |   0.03 |    0.00 |   0.00 |    4.53 |    571.33 | 72,839,589 | 58,643 |   3,062 | exactly | analogous     |
| **exactly~perpendicular** |    50 |      1.00 |       49.00 |    0.04 |   0.04 |    0.00 |   0.00 |    4.34 |    295.37 | 72,839,589 | 58,643 |   1,240 | exactly | perpendicular |
| **exactly~conducive**     |   208 |      7.33 |      200.67 |    0.02 |   0.02 |    0.00 |   0.00 |    4.25 |    995.32 | 72,839,589 | 58,643 |   9,110 | exactly | conducive     |


1. exactly_alike
   >  No two of your characters will be exactly alike .

2. exactly_opposite
   >  Similar laws are in place for the quark and antiquark that make up a meson : their respective colors must be exactly opposite .

3. exactly_right
   >  All that is exactly right , because deciding on a university is often the first really major decision of a young adult 's life .

4. exactly_sure
   >  Not exactly sure what a Prof is ?

5. exactly_zero
   >  I did sign up for a trip to Pai Canyon to watch the sunset , and was able to chat with a group of girls from the UK on the way up and the way back , so my social interaction was n't exactly zero for Day 1 solo on the roads .

6. exactly_parallel
   >  The most common obstacle is binding which can result when mounting surfaces are n't exactly parallel .

7. exactly_stellar
   >  It was a long day for them and the Browns are n't exactly stellar in the passing game .

8. exactly_analogous
   >  In short , the cases are almost exactly analogous .

9. exactly_perpendicular
   >  In this context , the rotation of the ground floor white volume is included in order to be located exactly perpendicular to the south .

10. exactly_conducive
   >  Not exactly conducive to being a rock star .

Saving Samples in /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/exactly/...

Samples saved as...
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/exactly/exactly_alike_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/exactly/exactly_opposite_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/exactly/exactly_right_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/exactly/exactly_sure_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/exactly/exactly_zero_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/exactly/exactly_parallel_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/exactly/exactly_stellar_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/exactly/exactly_analogous_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/exactly/exactly_perpendicular_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/exactly/exactly_conducive_50ex.csv

## *any*


|                 |    `f` |   `exp_f` |   `unexp_f` |   `dP1` |   `P1` |   `dP2` |   `P2` |   `LRC` |      `G2` |        `N` |   `f1` |    `f2` | `l1`   | `l2`    |
|:----------------|-------:|----------:|------------:|--------:|-------:|--------:|-------:|--------:|----------:|-----------:|-------:|--------:|:-------|:--------|
| **any~happier** |    963 |      7.84 |      955.16 |    0.06 |   0.06 |    0.03 |   0.03 |    6.75 |  7,438.55 | 72,839,589 | 34,382 |  16,606 | any    | happier |
| **any~clearer** |    608 |      5.51 |      602.49 |    0.05 |   0.05 |    0.02 |   0.02 |    6.51 |  4,556.17 | 72,839,589 | 34,382 |  11,680 | any    | clearer |
| **any~closer**  |  1,738 |     29.02 |    1,708.98 |    0.03 |   0.03 |    0.05 |   0.05 |    5.74 | 10,942.35 | 72,839,589 | 34,382 |  61,475 | any    | closer  |
| **any~wiser**   |    141 |      1.71 |      139.29 |    0.04 |   0.04 |    0.00 |   0.00 |    5.66 |    971.10 | 72,839,589 | 34,382 |   3,630 | any    | wiser   |
| **any~cuter**   |     80 |      0.86 |       79.14 |    0.04 |   0.04 |    0.00 |   0.00 |    5.56 |    570.11 | 72,839,589 | 34,382 |   1,828 | any    | cuter   |
| **any~safer**   |    626 |     10.77 |      615.23 |    0.03 |   0.03 |    0.02 |   0.02 |    5.56 |  3,883.22 | 72,839,589 | 34,382 |  22,826 | any    | safer   |
| **any~worse**   |  3,673 |     84.50 |    3,588.50 |    0.02 |   0.02 |    0.10 |   0.11 |    5.33 | 20,994.30 | 72,839,589 | 34,382 | 179,012 | any    | worse   |
| **any~better**  | 11,460 |    295.36 |   11,164.64 |    0.02 |   0.02 |    0.32 |   0.33 |    5.23 | 65,861.94 | 72,839,589 | 34,382 | 625,733 | any    | better  |
| **any~truer**   |     56 |      0.67 |       55.33 |    0.04 |   0.04 |    0.00 |   0.00 |    5.16 |    386.71 | 72,839,589 | 34,382 |   1,427 | any    | truer   |
| **any~nearer**  |     66 |      0.87 |       65.13 |    0.04 |   0.04 |    0.00 |   0.00 |    5.15 |    444.12 | 72,839,589 | 34,382 |   1,836 | any    | nearer  |


1. any_happier
   >  Even though he arrived a week before my due date Frankie weighed 7lb 8oz , was in perfect health and we could n't have been any happier !

2. any_clearer
   >  The dramaturgy cannot be any clearer in trying to highlight where our focus should be .

3. any_closer
   >  Asked if she was any closer to giving an endorsement , Palin told the Fox News host that she could only tell him what she 'd do if she were a South Carolinian .

4. any_wiser
   >  Is burning binary bits of information any wiser ?

5. any_cuter
   >  It could n't be any cuter .

6. any_safer
   >  In Kate Thompson 's Creature of the Night Bobby 's mum is running away , taking her family with her , but is the country any safer than the city ? *

7. any_worse
   >  " Can this day get any worse ? "

8. any_better
   >  Do you think his odds would have been any better if he 'd been publishing his work today ?

9. any_truer
   >  In the case of Alan , this saying could not have been any truer .

10. any_nearer
   >  No one lives any nearer than town .

Saving Samples in /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/any/...

Samples saved as...
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/any/any_happier_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/any/any_clearer_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/any/any_closer_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/any/any_wiser_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/any/any_cuter_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/any/any_safer_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/any/any_worse_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/any/any_better_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/any/any_truer_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/any/any_nearer_50ex.csv

## *remotely*


|                          |   `f` |   `exp_f` |   `unexp_f` |   `dP1` |   `P1` |   `dP2` |   `P2` |   `LRC` |     `G2` |        `N` |   `f1` |    `f2` | `l1`     | `l2`        |
|:-------------------------|------:|----------:|------------:|--------:|-------:|--------:|-------:|--------:|---------:|-----------:|-------:|--------:|:---------|:------------|
| **remotely~detonated**   |    78 |      0.02 |       77.98 |    0.88 |   0.88 |    0.00 |   0.00 |   12.69 | 1,243.75 | 72,839,589 | 16,426 |      89 | remotely | detonated   |
| **remotely~exploitable** |   145 |      0.22 |      144.78 |    0.15 |   0.15 |    0.01 |   0.01 |    8.79 | 1,613.38 | 72,839,589 | 16,426 |     986 | remotely | exploitable |
| **remotely~comparable**  |   277 |      2.76 |      274.24 |    0.02 |   0.02 |    0.02 |   0.02 |    6.15 | 2,015.00 | 72,839,589 | 16,426 |  12,252 | remotely | comparable  |
| **remotely~plausible**   |   183 |      3.96 |      179.04 |    0.01 |   0.01 |    0.01 |   0.01 |    4.89 | 1,048.46 | 72,839,589 | 16,426 |  17,571 | remotely | plausible   |
| **remotely~related**     |   617 |     23.76 |      593.24 |    0.01 |   0.01 |    0.04 |   0.04 |    4.36 | 2,857.41 | 72,839,589 | 16,426 | 105,375 | remotely | related     |
| **remotely~close**       | 1,597 |     92.76 |    1,504.24 |    0.00 |   0.00 |    0.09 |   0.10 |    3.90 | 6,229.80 | 72,839,589 | 16,426 | 411,329 | remotely | close       |
| **remotely~interested**  | 1,062 |     59.65 |    1,002.35 |    0.00 |   0.00 |    0.06 |   0.06 |    3.90 | 4,177.56 | 72,839,589 | 16,426 | 264,528 | remotely | interested  |
| **remotely~credible**    |   100 |      3.68 |       96.32 |    0.01 |   0.01 |    0.01 |   0.01 |    3.87 |   468.95 | 72,839,589 | 16,426 |  16,318 | remotely | credible    |
| **remotely~possible**    |   727 |     55.31 |      671.69 |    0.00 |   0.00 |    0.04 |   0.04 |    3.41 | 2,431.85 | 72,839,589 | 16,426 | 245,272 | remotely | possible    |
| **remotely~believable**  |    68 |      3.12 |       64.88 |    0.00 |   0.00 |    0.00 |   0.00 |    3.33 |   290.03 | 72,839,589 | 16,426 |  13,823 | remotely | believable  |


1. remotely_detonated
   >  the bomb hit the car only a few miles from its destination , officials said , and may have been remotely detonated .

2. remotely_exploitable
   >  Sun Systems products get 23 fixes , seven of which are remotely exploitable .

3. remotely_comparable
   >  Yes , yes , I know , the cases are not remotely comparable .

4. remotely_plausible
   >  Which reminds me , how was it even remotely plausible that Sally Yates was convinced that Mike Flynn was compromised because he had n't been completely forthright in disclosing his conversation with the Russian ambassador ?

5. remotely_related
   >  If you like Treme and restaurants , or restaurants only , or Bourdain or , well , any other things even remotely related to the whole thing , read it .

6. remotely_close
   >  " In only one case did I find anything even remotely close where an officer wrote that the victim did not resist to the best of her ability . "

7. remotely_interested
   >  Since the game is free-to - play , anybody who 's even remotely interested in tank warfare or shooting down birds out of the sky , would be missing out should they choose not give it a go .

8. remotely_credible
   >  Michiko Kakutani , writing in The New York Times , bridled at the book 's `` grotesque , voyeuristic scenes '' and found the female characters not `` remotely credible . ''

9. remotely_possible
   >  The guy getting credit for revealing the fraud actually did a lot of work that is not remotely possible for the average retail investor , or even the extraordinary retail investor , including going to China to meet the company .

10. remotely_believable
   >  although there is nothing remotely believable about this drawn-out cat-and-mouse game of a movie crossed with a whodunit , that 's almost the point .

Saving Samples in /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/remotely/...

Samples saved as...
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/remotely/remotely_detonated_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/remotely/remotely_exploitable_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/remotely/remotely_comparable_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/remotely/remotely_plausible_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/remotely/remotely_related_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/remotely/remotely_close_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/remotely/remotely_interested_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/remotely/remotely_credible_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/remotely/remotely_possible_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/remotely/remotely_believable_50ex.csv

## *ever*


|                      |   `f` |   `exp_f` |   `unexp_f` |   `dP1` |   `P1` |   `dP2` |   `P2` |   `LRC` |      `G2` |        `N` |    `f1` |   `f2` | `l1`   | `l2`        |
|:---------------------|------:|----------:|------------:|--------:|-------:|--------:|-------:|--------:|----------:|-----------:|--------:|-------:|:-------|:------------|
| **ever~olympic**     |   218 |      0.77 |      217.23 |    0.44 |   0.44 |    0.00 |   0.00 |    8.23 |  2,141.80 | 72,839,589 | 114,075 |    492 | ever   | olympic     |
| **ever~quarterly**   |   137 |      0.48 |      136.52 |    0.45 |   0.45 |    0.00 |   0.00 |    8.05 |  1,349.65 | 72,839,589 | 114,075 |    306 | ever   | quarterly   |
| **ever~watchful**    |   416 |      2.92 |      413.08 |    0.22 |   0.22 |    0.00 |   0.00 |    7.05 |  3,399.87 | 72,839,589 | 114,075 |  1,866 | ever   | watchful    |
| **ever~closer**      | 6,307 |     96.28 |    6,210.72 |    0.10 |   0.10 |    0.05 |   0.06 |    6.08 | 41,328.73 | 72,839,589 | 114,075 | 61,475 | ever   | closer      |
| **ever~joint**       |   195 |      2.29 |      192.71 |    0.13 |   0.13 |    0.00 |   0.00 |    5.95 |  1,375.51 | 72,839,589 | 114,075 |  1,460 | ever   | joint       |
| **ever~nearer**      |   223 |      2.88 |      220.12 |    0.12 |   0.12 |    0.00 |   0.00 |    5.84 |  1,528.28 | 72,839,589 | 114,075 |  1,836 | ever   | nearer      |
| **ever~vigilant**    |   923 |     14.94 |      908.06 |    0.10 |   0.10 |    0.01 |   0.01 |    5.80 |  5,892.45 | 72,839,589 | 114,075 |  9,541 | ever   | vigilant    |
| **ever~diminishing** |    71 |      0.70 |       70.30 |    0.16 |   0.16 |    0.00 |   0.00 |    5.75 |    527.77 | 72,839,589 | 114,075 |    445 | ever   | diminishing |
| **ever~scarcer**     |    82 |      0.85 |       81.15 |    0.15 |   0.15 |    0.00 |   0.00 |    5.75 |    599.19 | 72,839,589 | 114,075 |    545 | ever   | scarcer     |
| **ever~shrinking**   |   120 |      1.55 |      118.45 |    0.12 |   0.12 |    0.00 |   0.00 |    5.60 |    822.02 | 72,839,589 | 114,075 |    989 | ever   | shrinking   |


1. ever_olympic
   >  Naidan Tuvshinbayar won Mongolia 's first ever Olympic gold medal when claiming the men's 100 kg title .

2. ever_quarterly
   >  Losses of Tw $ 9.8 billion in the last three months of 2017 represented its worst ever quarterly results .

3. ever_watchful
   >  Owls are a rowdy bunch with fantasy colored feathers in bright oranges , greens , reds , yellows , blues and purples , with feathers that stick out in all directions and their signature yellow eyes ever watchful for an opportunity to clown around .

4. ever_closer
   >  CW : Since the US / EU stance on the two above-territories since the collapse of the USSR has effectively achieved the precise opposite of what that stance was designed to achieve ( by pushing the regions into ever closer ties with Russia ) , how can there be any logical defence of continuing that policy ?

5. ever_joint
   >  The Nepal Army has said that Kathmandu and Beijing will conduct their first ever joint military exercise from April 16 .

6. ever_nearer
   >  At Wednesday 's ceremony , the 2012 medals will be shown to the world for the first time , yet another reminder that the Games are drawing ever nearer .

7. ever_vigilant
   >  " One after the other , each of us must be ever vigilant , to relentlessly overcome each challenge , to reach higher , set more innovative goals , and bring joy to our customers .

8. ever_diminishing
   >  Miners create a block after a time frame which is worth an ever diminishing amount of currency or some kind of reward in order to ensure the shortfall .

9. ever_scarcer
   >  As The City 's rents rise even higher and real estate becomes ever scarcer with more people flocking here to take advantage of the tech -fueled economic boom , a new kind of street person is emerging : older , gay , and living with HIV or AIDS .

10. ever_shrinking
   >  In our complicated and ever shrinking world , the power of different bodies to make or administer law is often unclear .

Saving Samples in /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/ever/...

Samples saved as...
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/ever/ever_olympic_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/ever/ever_quarterly_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/ever/ever_watchful_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/ever/ever_closer_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/ever/ever_joint_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/ever/ever_nearer_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/ever/ever_vigilant_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/ever/ever_diminishing_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/ever/ever_scarcer_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/ever/ever_shrinking_50ex.csv

## *yet*


|                      |   `f` |   `exp_f` |   `unexp_f` |   `dP1` |   `P1` |   `dP2` |   `P2` |   `LRC` |     `G2` |        `N` |   `f1` |   `f2` | `l1`   | `l2`         |
|:---------------------|------:|----------:|------------:|--------:|-------:|--------:|-------:|--------:|---------:|-----------:|-------:|-------:|:-------|:-------------|
| **yet~unborn**       |   372 |      0.68 |      371.32 |    0.72 |   0.72 |    0.00 |   0.00 |   10.14 | 4,319.00 | 72,839,589 | 95,763 |    519 | yet    | unborn       |
| **yet~unnamed**      |   736 |      2.77 |      733.23 |    0.35 |   0.35 |    0.01 |   0.01 |    8.29 | 7,048.15 | 72,839,589 | 95,763 |  2,107 | yet    | unnamed      |
| **yet~unspecified**  |   204 |      0.80 |      203.20 |    0.33 |   0.34 |    0.00 |   0.00 |    7.86 | 1,933.20 | 72,839,589 | 95,763 |    607 | yet    | unspecified  |
| **yet~undetermined** |   297 |      1.45 |      295.55 |    0.27 |   0.27 |    0.00 |   0.00 |    7.55 | 2,658.03 | 72,839,589 | 95,763 |  1,104 | yet    | undetermined |
| **yet~untitled**     |   121 |      0.54 |      120.46 |    0.29 |   0.29 |    0.00 |   0.00 |    7.37 | 1,107.51 | 72,839,589 | 95,763 |    412 | yet    | untitled     |
| **yet~unidentified** |   336 |      2.48 |      333.52 |    0.18 |   0.18 |    0.00 |   0.00 |    6.85 | 2,694.33 | 72,839,589 | 95,763 |  1,890 | yet    | unidentified |
| **yet~unformed**     |    53 |      0.33 |       52.67 |    0.21 |   0.21 |    0.00 |   0.00 |    6.28 |   445.94 | 72,839,589 | 95,763 |    249 | yet    | unformed     |
| **yet~final**        |   643 |      7.70 |      635.30 |    0.11 |   0.11 |    0.01 |   0.01 |    6.20 | 4,494.99 | 72,839,589 | 95,763 |  5,860 | yet    | final        |
| **yet~unannounced**  |   129 |      1.16 |      127.84 |    0.14 |   0.15 |    0.00 |   0.00 |    6.19 |   979.33 | 72,839,589 | 95,763 |    883 | yet    | unannounced  |
| **yet~undiscovered** |   278 |      2.99 |      275.01 |    0.12 |   0.12 |    0.00 |   0.00 |    6.18 | 2,006.07 | 72,839,589 | 95,763 |  2,272 | yet    | undiscovered |


1. yet_unborn
   >  You may spill your blood as a purely symbolic gesture , in service to those humans yet unborn .

2. yet_unnamed
   >  That will be powered by a yet unnamed 1.6 Ghz octa-core processor , supported by 2GB RAM .

3. yet_unspecified
   >  most of the justices are troubled by outsized verdicts , and the court has ruled that the constitutional guarantee of due process of law places some , as yet unspecified , limits on punitive damages .

4. yet_undetermined
   >  Clashes had five wounded Saturday night in circumstances yet undetermined .

5. yet_untitled
   >  Actor-filmmaker Kamal Haasan will reportedly join hands with his erstwhile assistant Rajesh M Selva , director of his last Tamil outing " Thoongaavanam " , for a yet untitled Tamil actioner .

6. yet_unidentified
   >  They are focusing genetic testing on families , like Crissy 's , where breast cancer is hereditary and develops from a yet unidentified gene mutation .

7. yet_unformed
   >  Whilst earth , as yet unformed and void ,

8. yet_final
   >  The duties are not yet final , so we will continue to use all the means available to us to ensure a fair outcome to these investigations , " the sources explained .

9. yet_unannounced
   >  3 . A yet unannounced candidate takes the White House

10. yet_undiscovered
   >  `` Countless seabirds , dolphins , fishes , corals and tiny things as yet undiscovered could survive as a result , free of the threats that are eliminating them elsewhere . ''

Saving Samples in /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/yet/...

Samples saved as...
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/yet/yet_unborn_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/yet/yet_unnamed_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/yet/yet_unspecified_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/yet/yet_undetermined_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/yet/yet_untitled_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/yet/yet_unidentified_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/yet/yet_unformed_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/yet/yet_final_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/yet/yet_unannounced_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/yet/yet_undiscovered_50ex.csv

## *immediately*


|                              |    `f` |   `exp_f` |   `unexp_f` |   `dP1` |   `P1` |   `dP2` |   `P2` |   `LRC` |       `G2` |        `N` |   `f1` |    `f2` | `l1`        | `l2`         |
|:-----------------------------|-------:|----------:|------------:|--------:|-------:|--------:|-------:|--------:|-----------:|-----------:|-------:|--------:|:------------|:-------------|
| **immediately~accretive**    |    236 |      0.70 |      235.30 |    0.45 |   0.45 |    0.00 |   0.00 |    8.54 |   2,406.67 | 72,839,589 | 96,973 |     523 | immediately | accretive    |
| **immediately~appealable**   |     76 |      0.19 |       75.81 |    0.55 |   0.55 |    0.00 |   0.00 |    8.41 |     815.23 | 72,839,589 | 96,973 |     139 | immediately | appealable   |
| **immediately~adjacent**     |  1,595 |      6.33 |    1,588.67 |    0.33 |   0.34 |    0.02 |   0.02 |    8.31 |  15,089.64 | 72,839,589 | 96,973 |   4,756 | immediately | adjacent     |
| **immediately~apparent**     |  4,971 |     72.22 |    4,898.78 |    0.09 |   0.09 |    0.05 |   0.05 |    6.12 |  32,982.98 | 72,839,589 | 96,973 |  54,246 | immediately | apparent     |
| **immediately~clear**        | 26,038 |    464.92 |   25,573.08 |    0.07 |   0.07 |    0.26 |   0.27 |    5.86 | 167,884.92 | 72,839,589 | 96,973 | 349,214 | immediately | clear        |
| **immediately~actionable**   |    173 |      2.64 |      170.36 |    0.09 |   0.09 |    0.00 |   0.00 |    5.47 |   1,121.82 | 72,839,589 | 96,973 |   1,983 | immediately | actionable   |
| **immediately~recognizable** |  1,736 |     44.60 |    1,691.40 |    0.05 |   0.05 |    0.02 |   0.02 |    5.15 |   9,447.17 | 72,839,589 | 96,973 |  33,499 | immediately | recognizable |
| **immediately~available**    | 29,351 |    887.87 |   28,463.13 |    0.04 |   0.04 |    0.29 |   0.30 |    5.06 | 159,088.56 | 72,839,589 | 96,973 | 666,909 | immediately | available    |
| **immediately~subsequent**   |     59 |      0.96 |       58.04 |    0.08 |   0.08 |    0.00 |   0.00 |    4.81 |     374.58 | 72,839,589 | 96,973 |     722 | immediately | subsequent   |
| **immediately~recognisable** |    349 |     10.92 |      338.08 |    0.04 |   0.04 |    0.00 |   0.00 |    4.59 |   1,757.24 | 72,839,589 | 96,973 |   8,204 | immediately | recognisable |


1. immediately_accretive
   >  " Further , we expect to leverage our purchasing , sales , distribution and administrative capabilities to improve the profitability of this business , and we expect this acquisition to be immediately accretive to Drew 's earnings . "

2. immediately_appealable
   >  Generally , the denial of a motion to dismiss under Rule 12 ( b ) ( 6 ) , SCRCP , is not immediately appealable .

3. immediately_adjacent
   >  Sometimes the parking is in its own big ugly building , and sometimes it 's designed so that it 's immediately adjacent to the apartment or townhouse of its occupant .

4. immediately_apparent
   >  " Research can be hard , and its benefits are n't always immediately apparent , " she shares .

5. immediately_clear
   >  the cause of the fire was not immediately clear but investigators say they believe it was an accident .

6. immediately_actionable
   >  I facilitate teacher workshops and online courses that deliver practical and immediately actionable advice on teachers from early childhood to Primary can develop student ICT capability in their lessons .

7. immediately_recognizable
   >  It 's an appropriate motto for the hitmaker savant , who has managed to fit a number of wildly disparate elements under that pop umbrella , crafting an immediately recognizable sound out of a multitude of genres .

8. immediately_available
   >  Financial details of the agreement were not immediately available , but Simon was more than satisfied .

9. immediately_subsequent
   >  At issue , as FHQ has discussed , is the discrepancy between the longstanding New Hampshire election law that requires seven days between the primary in the Granite state and the immediately subsequent primary or caucus and a newly -enacted Nevada Republican Party resolution tethering the party 's caucuses to the Saturday following the New Hampshire primary .

10. immediately_recognisable
   >  Its vastness and power have supplied immediately recognisable symbols for moral and religious works of art .

Saving Samples in /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/immediately/...

Samples saved as...
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/immediately/immediately_accretive_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/immediately/immediately_appealable_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/immediately/immediately_adjacent_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/immediately/immediately_apparent_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/immediately/immediately_clear_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/immediately/immediately_actionable_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/immediately/immediately_recognizable_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/immediately/immediately_available_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/immediately/immediately_subsequent_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/immediately/immediately_recognisable_50ex.csv

## *particularly*


|                              |   `f` |   `exp_f` |   `unexp_f` |   `dP1` |   `P1` |   `dP2` |   `P2` |   `LRC` |      `G2` |        `N` |    `f1` |   `f2` | `l1`         | `l2`        |
|:-----------------------------|------:|----------:|------------:|--------:|-------:|--------:|-------:|--------:|----------:|-----------:|--------:|-------:|:-------------|:------------|
| **particularly~hard-hit**    |   154 |      2.76 |      151.24 |    0.39 |   0.39 |    0.00 |   0.00 |    5.64 |  1,005.08 | 72,839,589 | 513,668 |    391 | particularly | hard-hit    |
| **particularly~galling**     |   540 |     15.33 |      524.67 |    0.24 |   0.25 |    0.00 |   0.00 |    5.12 |  2,937.15 | 72,839,589 | 513,668 |  2,174 | particularly | galling     |
| **particularly~well-suited** |    62 |      1.22 |       60.78 |    0.35 |   0.36 |    0.00 |   0.00 |    4.91 |    390.17 | 72,839,589 | 513,668 |    173 | particularly | well-suited |
| **particularly~acute**       | 2,809 |    109.23 |    2,699.77 |    0.17 |   0.18 |    0.01 |   0.01 |    4.79 | 13,361.56 | 72,839,589 | 513,668 | 15,489 | particularly | acute       |
| **particularly~noteworthy**  | 2,294 |    112.66 |    2,181.34 |    0.14 |   0.14 |    0.00 |   0.00 |    4.37 |  9,788.31 | 72,839,589 | 513,668 | 15,975 | particularly | noteworthy  |
| **particularly~thorny**      |   308 |     12.43 |      295.57 |    0.17 |   0.17 |    0.00 |   0.00 |    4.36 |  1,439.60 | 72,839,589 | 513,668 |  1,762 | particularly | thorny      |
| **particularly~fond**        | 4,179 |    229.28 |    3,949.72 |    0.12 |   0.13 |    0.01 |   0.01 |    4.24 | 16,897.66 | 72,839,589 | 513,668 | 32,513 | particularly | fond        |
| **particularly~nettlesome**  |    63 |      2.03 |       60.97 |    0.21 |   0.22 |    0.00 |   0.00 |    4.02 |    324.87 | 72,839,589 | 513,668 |    288 | particularly | nettlesome  |
| **particularly~nasty**       | 2,143 |    135.27 |    2,007.73 |    0.10 |   0.11 |    0.00 |   0.00 |    3.96 |  8,052.99 | 72,839,589 | 513,668 | 19,181 | particularly | nasty       |
| **particularly~egregious**   | 1,705 |    107.12 |    1,597.88 |    0.11 |   0.11 |    0.00 |   0.00 |    3.94 |  6,421.57 | 72,839,589 | 513,668 | 15,190 | particularly | egregious   |


1. particularly_hard-hit
   >  shares of financial stocks were particularly hard-hit .

2. particularly_galling
   >  the brazen presence in Poti has been particularly galling for Georgia because it is hundreds of kilometers -LRB- miles -RRB- from South Ossetia , where the war broke out and where most of the fighting occurred .

3. particularly_well-suited
   >  but the plutonium produced by its two breeders , Monju and Joyo , is a kind that is particularly well-suited to bombs , Greenpeace said .

4. particularly_acute
   >  The risks were particularly acute for correspondents in the north and east and those covering political events .

5. particularly_noteworthy
   >  Bushey recalled how , shortly after learning of what he called `` a particularly noteworthy crime , '' he directed one of his subordinates to inform Simpson `` just as soon as was humanly possible . ''

6. particularly_thorny
   >  With $ 15,000 payouts for first place and particularly thorny fields , the Championships have traditionally been considered among pool 's domestic majors .

7. particularly_fond
   >  For example , I love mixing the herbal Honeybush Vanilla [ 19 ] with a black English Breakfast [ 20 ] tea. Adagio [ 21 ] allows users to upload their favourite blends so that other tea drinkers can purchase them ( I am particularly fond of the Captain America [ 22 ] and TARDIS [ 23 ] blends ) .

8. particularly_nettlesome
   >  The court in East Bridge Lofts described that issue as " particularly nettlesome , " but it also found that South Carolina ( whose law governed this issue ) applies what is known as the " automatic waiver " rule :

9. particularly_nasty
   >  Hunter , known for his in-your-face style , gave a particularly nasty time to Red Wings captain Steve Yzerman .

10. particularly_egregious
   >  Veldhuis ' case is particularly egregious , because he was fired while his classes were still in session .

Saving Samples in /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/particularly/...

Samples saved as...
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/particularly/particularly_hard-hit_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/particularly/particularly_galling_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/particularly/particularly_well-suited_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/particularly/particularly_acute_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/particularly/particularly_noteworthy_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/particularly/particularly_thorny_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/particularly/particularly_fond_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/particularly/particularly_nettlesome_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/particularly/particularly_nasty_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/particularly/particularly_egregious_50ex.csv

## *inherently*


|                                  |   `f` |   `exp_f` |   `unexp_f` |   `dP1` |   `P1` |   `dP2` |   `P2` |   `LRC` |     `G2` |        `N` |   `f1` |   `f2` | `l1`       | `l2`              |
|:---------------------------------|------:|----------:|------------:|--------:|-------:|--------:|-------:|--------:|---------:|-----------:|-------:|-------:|:-----------|:------------------|
| **inherently~governmental**      |   253 |      0.50 |      252.50 |    0.33 |   0.33 |    0.01 |   0.01 |    8.92 | 2,742.59 | 72,839,589 | 47,803 |    761 | inherently | governmental      |
| **inherently~unequal**           |   390 |      3.69 |      386.31 |    0.07 |   0.07 |    0.01 |   0.01 |    6.38 | 2,893.14 | 72,839,589 | 47,803 |  5,621 | inherently | unequal           |
| **inherently~evil**              | 1,123 |     14.90 |    1,108.10 |    0.05 |   0.05 |    0.02 |   0.02 |    6.05 | 7,572.62 | 72,839,589 | 47,803 | 22,706 | inherently | evil              |
| **inherently~coercive**          |    94 |      0.82 |       93.18 |    0.07 |   0.08 |    0.00 |   0.00 |    5.98 |   711.86 | 72,839,589 | 47,803 |  1,253 | inherently | coercive          |
| **inherently~unstable**          |   806 |     14.80 |      791.20 |    0.04 |   0.04 |    0.02 |   0.02 |    5.52 | 4,903.18 | 72,839,589 | 47,803 | 22,546 | inherently | unstable          |
| **inherently~sinful**            |   145 |      2.01 |      142.99 |    0.05 |   0.05 |    0.00 |   0.00 |    5.49 |   962.39 | 72,839,589 | 47,803 |  3,059 | inherently | sinful            |
| **inherently~flawed**            |   899 |     18.13 |      880.87 |    0.03 |   0.03 |    0.02 |   0.02 |    5.39 | 5,301.74 | 72,839,589 | 47,803 | 27,628 | inherently | flawed            |
| **inherently~untrustworthy**     |    69 |      0.79 |       68.21 |    0.06 |   0.06 |    0.00 |   0.00 |    5.39 |   483.61 | 72,839,589 | 47,803 |  1,211 | inherently | untrustworthy     |
| **inherently~interdisciplinary** |    94 |      1.25 |       92.75 |    0.05 |   0.05 |    0.00 |   0.00 |    5.35 |   631.33 | 72,839,589 | 47,803 |  1,906 | inherently | interdisciplinary |
| **inherently~discriminatory**    |   175 |      2.94 |      172.06 |    0.04 |   0.04 |    0.00 |   0.00 |    5.27 | 1,093.34 | 72,839,589 | 47,803 |  4,481 | inherently | discriminatory    |


1. inherently_governmental
   >  " When it comes to security in war zones , what is considered ' inherently governmental ? '

2. inherently_unequal
   >  And this is when the Supreme Court of the U.S. ruled that the " separate but equal " policy was inherently unequal .

3. inherently_evil
   >  The layperson 's affirmation of this proposition as necessarily true is linked essentially to some pretty basic " inside " feelings that rape is inherently evil .

4. inherently_coercive
   >  While direct contact is somewhat more intrusive , it is not inherently coercive .

5. inherently_unstable
   >  " Vitamin C is inherently unstable and easily oxidizes in air , which makes it ineffective .

6. inherently_sinful
   >  The position taken by most Anglican churches in Africa , which see homosexuality as inherently sinful ( at best ) is in the end irreconcilable with the liberal views which predominate in North America and increasingly ( though far from uniformly ) in the Church of England itself .

7. inherently_flawed
   >  A man's means may become his end as a result of conscious and reasoned choice , and there is nothing inherently flawed in this approach .

8. inherently_untrustworthy
   >  As such , UN agencies like the WTO , Human Rights Council , and World Bank are inevitably unethical and inherently untrustworthy .

9. inherently_interdisciplinary
   >  The problems are inherently interdisciplinary .

10. inherently_discriminatory
   >  some analysts said whites , often the chief architect of standardized tests and the products of better schools , tend to perform better than minorities _ making reliance on the test inherently discriminatory .

Saving Samples in /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/inherently/...

Samples saved as...
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/inherently/inherently_governmental_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/inherently/inherently_unequal_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/inherently/inherently_evil_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/inherently/inherently_coercive_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/inherently/inherently_unstable_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/inherently/inherently_sinful_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/inherently/inherently_flawed_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/inherently/inherently_untrustworthy_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/inherently/inherently_interdisciplinary_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/inherently/inherently_discriminatory_50ex.csv

## *terribly*


|                          |   `f` |   `exp_f` |   `unexp_f` |   `dP1` |   `P1` |   `dP2` |   `P2` |   `LRC` |      `G2` |        `N` |   `f1` |    `f2` | `l1`     | `l2`        |
|:-------------------------|------:|----------:|------------:|--------:|-------:|--------:|-------:|--------:|----------:|-----------:|-------:|--------:|:---------|:------------|
| **terribly~awry**        |   180 |      0.56 |      179.44 |    0.26 |   0.26 |    0.00 |   0.00 |    8.02 |  1,770.97 | 72,839,589 | 58,964 |     692 | terribly | awry        |
| **terribly~wrong**       | 6,349 |    120.67 |    6,228.33 |    0.04 |   0.04 |    0.11 |   0.11 |    5.67 | 38,814.14 | 72,839,589 | 58,964 | 149,064 | terribly | wrong       |
| **terribly~amiss**       |    64 |      0.73 |       63.27 |    0.07 |   0.07 |    0.00 |   0.00 |    5.37 |    451.25 | 72,839,589 | 58,964 |     898 | terribly | amiss       |
| **terribly~sorry**       | 1,262 |     50.65 |    1,211.35 |    0.02 |   0.02 |    0.02 |   0.02 |    4.43 |  5,741.83 | 72,839,589 | 58,964 |  62,573 | terribly | sorry       |
| **terribly~misguided**   |   101 |      3.24 |       97.76 |    0.02 |   0.03 |    0.00 |   0.00 |    4.09 |    501.57 | 72,839,589 | 58,964 |   4,008 | terribly | misguided   |
| **terribly~inefficient** |   198 |      7.89 |      190.11 |    0.02 |   0.02 |    0.00 |   0.00 |    4.05 |    900.41 | 72,839,589 | 58,964 |   9,744 | terribly | inefficient |
| **terribly~surprising**  |   964 |     57.10 |      906.90 |    0.01 |   0.01 |    0.02 |   0.02 |    3.82 |  3,660.97 | 72,839,589 | 58,964 |  70,540 | terribly | surprising  |
| **terribly~sad**         | 1,241 |     80.21 |    1,160.79 |    0.01 |   0.01 |    0.02 |   0.02 |    3.73 |  4,513.50 | 72,839,589 | 58,964 |  99,081 | terribly | sad         |
| **terribly~unfair**      |   367 |     20.86 |      346.14 |    0.01 |   0.01 |    0.01 |   0.01 |    3.71 |  1,419.19 | 72,839,589 | 58,964 |  25,769 | terribly | unfair      |
| **terribly~frightened**  |   137 |      6.77 |      130.23 |    0.02 |   0.02 |    0.00 |   0.00 |    3.60 |    565.89 | 72,839,589 | 58,964 |   8,364 | terribly | frightened  |


1. terribly_awry
   >  As tends to happen , the experiment goes terribly awry .

2. terribly_wrong
   >  But he spoke with the confidence of someone who thinks his country has got things right , while others have got them terribly wrong .

3. terribly_amiss
   >  " It was made blatantly obvious to the millions of people watching the post parade that something was terribly amiss ( with Life At Ten ) , " De Bartolo 's letter said .

4. terribly_sorry
   >  I am terribly sorry to have to leave you after knowing you for such a short time .

5. terribly_misguided
   >  well , he 's a genius again , though I fear a terribly misguided one .

6. terribly_inefficient
   >  Well , I did write , but it was terribly inefficient to pile up the letters telling you I was drinking and studying and talking with folks in all sorts of interesting tongues .

7. terribly_surprising
   >  This story is a few days old ( and probably not terribly surprising ) , but I just wanted to mention Indiana University professor Julia R. Fox 's study that concludes that The Daily Show is just as substantive as nightly network news .

8. terribly_sad
   >  " Although I am terribly sad that we just lost her , I think that I am privileged to have known her , " Suen said .

9. terribly_unfair
   >  It ca n't be terribly unfair to the United States if there is zero consequence for the United States not meeting its targets .

10. terribly_frightened
   >  We are now terribly angry at your Patient ; now that we are no longer so terribly frightened about him !

Saving Samples in /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/terribly/...

Samples saved as...
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/terribly/terribly_awry_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/terribly/terribly_wrong_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/terribly/terribly_amiss_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/terribly/terribly_sorry_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/terribly/terribly_misguided_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/terribly/terribly_inefficient_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/terribly/terribly_surprising_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/terribly/terribly_sad_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/terribly/terribly_unfair_50ex.csv
+ /share/compling/projects/sanpi/results/top_AM/any_bigram_examples/terribly/terribly_frightened_50ex.csv
